In [20]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext , Document , Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
from PyPDF2 import PdfReader
from IPython.display import Markdown, display
import chromadb
import os
from dotenv import load_dotenv
import uuid
import json
load_dotenv()

True

## Define função que copia os documentos para o diretório do ChromaDB

In [ ]:
import shutil
import os


def move_docs_to_chroma_folder(diretorio_origem,collection_name):

    # Diretório de destino
    diretorio_destino = f'chroma_db/{collection_name}_files'

    # Verifica se o diretório de destino existe, se não, cria
    if not os.path.exists(diretorio_destino):
        os.makedirs(diretorio_destino)

    # Lista todos os arquivos no diretório de origem
    arquivos = os.listdir(diretorio_origem)

    # Copia cada arquivo para o diretório de destino
    for arquivo in arquivos:
        caminho_origem = os.path.join(diretorio_origem, arquivo)
        caminho_destino = os.path.join(diretorio_destino, arquivo)
        shutil.copy2(caminho_origem, caminho_destino)

## Carregando os documentos do diretório

In [3]:
collection_name = "livros"
embed_model_name = 'all-MiniLM-L6-v2'
folder_name = 'livros'


embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name=embed_model_name))

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


documents = SimpleDirectoryReader(folder_name).load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model , show_progress=True
)

move_docs_to_chroma_folder(folder_name,collection_name)

## Utilizando Semantic Chunking

In [11]:
collection_name = "docs-semantic-chunking"
embed_model_name = 'all-MiniLM-L6-v2'
folder_name = 'docs'


embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name=embed_model_name))

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


documents = SimpleDirectoryReader(folder_name).load_data()



base_splitter = SentenceSplitter(chunk_size=512)

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

nodes = splitter.get_nodes_from_documents(documents)

index = VectorStoreIndex(
    nodes=nodes, storage_context=storage_context, embed_model=embed_model,show_progress=True
)

move_docs_to_chroma_folder(folder_name,collection_name)